# Configuration
Import the fundamental libraries.

In [ ]:
# install cloud-tpu-client to use tpus
! pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.7-cp36-cp36m-linux_x86_64.whl

# install 🤗 Transformers library
! pip install transformers

ERROR: torch_xla-1.7-cp36-cp36m-linux_x86_64.whl is not a supported wheel on this platform.
     |████████████████████████████████| 2.0MB 19.9MB/s 
     |████████████████████████████████| 3.2MB 24.0MB/s 
     |████████████████████████████████| 890kB 52.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=16530ce05633845b826d1d50780d420b4c1c0efc142220512578e6524af3227c
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


Mount drive and copy the data and source code in local folder.

In [ ]:
# mount drive
from google.colab import drive
drive.mount('/content/drive')

# copy the files from drive
!cp -R -a '/content/drive/MyDrive/Colab Notebooks/NLP/SQUAD Project/cli_version/.' '/content/'

Mounted at /content/drive


# Training
Split the set in training, validation and testing. Edit the configuration file for the split percentages.

In [ ]:
# unzip the content of the zip file from google drive into a local folder "data"
!unzip -o -j '/content/data/SQUAD MATERIAL.zip' 'training_set.json' -d '/content/data/'

# create the three splits
!python3 create_splits.py data/training_set.json config.json

Archive:  /content/data/SQUAD MATERIAL.zip
  inflating: /content/data/training_set.json  
Traceback (most recent call last):
  File "create_splits.py", line 6, in <module>
    from src.dataset import SQuADJSONDataset
  File "/content/src/dataset.py", line 7, in <module>
    import torch
  File "/usr/local/lib/python3.7/dist-packages/torch/__init__.py", line 196, in <module>
    from torch._C import *
RuntimeError: KeyboardInterrupt: 


Run the following if you want to train a model. Edit the config.json to change hyperparameters and pre-trained model.

In [ ]:
# run the training
!python3 train.py data/train.json data/val.json --test data/test.json config.json

# copy the trained model to the models local folder
!unzip -o /content/experiments/*.zip *.pytorch -d /content/models/


>>> Using cuda device

>>> Loading configuration...
{'pretrained_model': 'albert-base-v1', 'type': 'enhanced2', 'batch_size': 16, 'num_epochs': 1, 'learning_rate': 0.0001, 'dropout_rate': 0.3, 'verbose': True, 'save_experiment': True, 'val_split': 0.15, 'test_split': 0.15}

>>> Loading the training split...
100% 308/308 [00:00<00:00, 5920.82argument/s]
>>> Loading the validation split...
100% 67/67 [00:00<00:00, 6801.19argument/s]
>>> Loading the testing split...
100% 67/67 [00:00<00:00, 5946.85argument/s]

>>> Configuring experimentation...
2021-03-08 18:37:57.044602: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
Downloading: 100% 684/684 [00:00<00:00, 963kB/s]
Downloading: 100% 47.4M/47.4M [00:00<00:00, 57.0MB/s]
Downloading: 100% 760k/760k [00:00<00:00, 27.4MB/s]
Downloading: 100% 1.31M/1.31M [00:00<00:00, 28.0MB/s]

>>> Epoch 1...
>>> Training...
  0% 5/3869 [00:08<1:44:51,  1.63s/batch]
Traceback (most recent

# Testing
Select one of the pre-trained models. DON'T RUN IF YOU WANT TO USE THE BEST MODEL.

In [ ]:
!bash /content/choose_model.sh

Choose one of the following models:
1) /content/models/bert-base-uncased-baseline-model.pytorch
2) /content/models/albert-base-v1-enhanced2-model.pytorch
3) /content/models/distilbert-base-uncased-baseline-model.pytorch
4) /content/models/albert-base-v1-baseline-model.pytorch
5) /content/models/albert-base-v1-enhanced1-model.pytorch
6) Stop the script
#? 1
Model /content/models/bert-base-uncased-baseline-model.pytorch selected


Execute the testing.

In [ ]:
!python3 compute_answers.py data/test_no_answers.json


>>> Using cuda device

>>> Loading the test split...

>>> Creating DataFrame...
100% 67/67 [00:00<00:00, 11608.01argument/s]

>>> Loading trained model...
Downloading: 100% 433/433 [00:00<00:00, 715kB/s]
Downloading: 100% 440M/440M [00:07<00:00, 62.3MB/s]
>>> Using model BaselineSQuADModel with bert-base-uncased pre-trained

>>> Loading tokenizer...
Downloading: 100% 232k/232k [00:00<00:00, 19.5MB/s]
Downloading: 100% 466k/466k [00:00<00:00, 19.0MB/s]
Downloading: 100% 28.0/28.0 [00:00<00:00, 41.9kB/s]

>>> Predicting answers for 13200 samples...
  0% 2/413 [00:02<09:01,  1.32s/batch]
Traceback (most recent call last):
  File "compute_answers.py", line 168, in <module>
    main()
  File "compute_answers.py", line 149, in main
    start_indexes = F.log_softmax(start_logits, dim=-1).detach().cpu()
KeyboardInterrupt


Compute scores.

In [ ]:
!python3 evaluate.py data/test.json data/predictions.json

{
  "exact": 63.70454545454545,
  "f1": 77.77324827577714,
  "total": 13200,
  "HasAns_exact": 63.70454545454545,
  "HasAns_f1": 77.77324827577714,
  "HasAns_total": 13200
}


Do the same for toy test.

In [ ]:
!python3 compute_answers.py /content/data/toy_test_no_answers.json
!python3 evaluate.py /content/data/toy_test.json data/predictions.json


>>> Using cuda device

>>> Loading the test split...

>>> Creating DataFrame...
100% 1/1 [00:00<00:00, 9078.58argument/s]

>>> Loading trained model...
>>> Using model EnhancedSQuADModel2 with albert-base-v1 pre-trained

>>> Loading tokenizer...

>>> Predicting answers for 2 samples...
100% 1/1 [00:00<00:00,  3.33batch/s]
{
  "exact": 100.0,
  "f1": 100.0,
  "total": 2,
  "HasAns_exact": 100.0,
  "HasAns_f1": 100.0,
  "HasAns_total": 2
}
